In [1]:
import pickle
import os
from sklearn.model_selection import KFold
import torch
import numpy as np
from models.bnkf import BNKF
from helper.evaluator import Evaluator

In [2]:
# training params
incl_rr = True

X_columns = ["measured_x", "measured_y", "measured_z", "measured_vx", "measured_vy", "measured_vz",
             "measured_plus_x", "measured_plus_y", "measured_plus_z", "measured_plus_vx", "measured_plus_vy", "measured_plus_vz", "sigma_range", "sigma_range_rate", "sigma_elevation", "sigma_bearing", "delta_time"]

y_columns = ["truth_x", "truth_y", "truth_z", "truth_vx", "truth_vy", "truth_vz"]

In [3]:
def construct_dict_folds(full_trajectory_dict, k=5):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    indices = []
    keys = list(full_trajectory_dict.keys())
    
    for train_indices, test_indices in kf.split(keys):
        train_keys = [keys[i] for i in train_indices]
        test_keys = [keys[i] for i in test_indices]
        indices.append((train_keys, test_keys))
    return indices

In [4]:
def return_index_cv_params(full_proc_traj_data_in, cv_indices):
    train_X_mats = []
    train_y_vecs = []
    test_X_mats = []
    test_y_vecs = []
    
    physics_metadata_train_dicts = []
    physics_metadata_test_dicts = []
    
    for index in cv_indices:
    
        train_pmd_dict = {
            'inputPos': torch.cat([full_proc_traj_data_in[key]['physics_meta_data']['inputPos'] for key in index[0]], dim=0),
            'inputVel': torch.cat([full_proc_traj_data_in[key]['physics_meta_data']['inputVel'] for key in index[0]], dim=0),
            'inputObs': torch.cat([full_proc_traj_data_in[key]['physics_meta_data']['inputObs'] for key in index[0]], dim=0),
            'inputRngErr': torch.cat([full_proc_traj_data_in[key]['physics_meta_data']['inputRngErr'] for key in index[0]], dim=0),
            'dtVec': torch.cat([full_proc_traj_data_in[key]['physics_meta_data']['dtVec'] for key in index[0]], dim=0)}
        
        train_X_mats.append(torch.cat([full_proc_traj_data_in[key]['X_mat'] for key in index[0]], dim=0))
        train_y_vecs.append(torch.cat([full_proc_traj_data_in[key]['y_vec'] for key in index[0]], dim=0))
        physics_metadata_train_dicts.append(train_pmd_dict)
        
        test_pmd_dict = {
            'inputPos': torch.cat([full_proc_traj_data_in[key]['physics_meta_data']['inputPos'] for key in index[1]], dim=0),
            'inputVel': torch.cat([full_proc_traj_data_in[key]['physics_meta_data']['inputVel'] for key in index[1]], dim=0),
            'inputObs': torch.cat([full_proc_traj_data_in[key]['physics_meta_data']['inputObs'] for key in index[1]], dim=0),
            'inputRngErr': torch.cat([full_proc_traj_data_in[key]['physics_meta_data']['inputRngErr'] for key in index[1]], dim=0),
            'dtVec': torch.cat([full_proc_traj_data_in[key]['physics_meta_data']['dtVec'] for key in index[1]], dim=0)}
        
        test_X_mats.append(torch.cat([full_proc_traj_data_in[key]['X_mat'] for key in index[1]], dim=0))
        test_y_vecs.append(torch.cat([full_proc_traj_data_in[key]['y_vec'] for key in index[1]], dim=0))
        physics_metadata_test_dicts.append(test_pmd_dict)
        
    return train_X_mats, train_y_vecs, test_X_mats, test_y_vecs, physics_metadata_train_dicts, physics_metadata_test_dicts

In [5]:
def evaluate_model_outputs(y_pred_state, y_pred_covs, y_truth_state):
    TestEval = Evaluator()
        
    full_pred_pos = y_pred_state[:, 0:3]
    full_truth_pos = y_truth_state[:, 0:3].cpu().numpy()
    
    proc_covariances = []    
    for idx, cov_vec in enumerate(y_pred_covs):
        proc_covariances.append(y_pred_covs[idx, 0:3, 0:3])
        
    avg_err = TestEval.calculate_3d_avg_euclid_error(full_truth_pos[:, 0], full_truth_pos[:, 1], full_truth_pos[:, 2], full_pred_pos[:, 0], full_pred_pos[:, 1], full_pred_pos[:, 2])

    avg_md = TestEval.average_mahalanobis_distance(full_pred_pos, proc_covariances, full_truth_pos)
        
    avg_det = TestEval.compute_average_determinant(proc_covariances)
    
    avg_trace = TestEval.compute_average_covariance_trace(proc_covariances)
    
    return avg_err, avg_md, avg_trace, avg_det

In [6]:
def cross_validate_pinn_model(full_proc_traj_data_in, cv_folds_in, sensor_noise_in='mid', show_info=False, epochs_in=1000, single_split=False):
    train_X_mats_cv, train_y_vecs_cv, test_X_mats_cv, test_y_vecs_cv, physics_metadata_train_dicts_cv, physics_metadata_test_dicts_cv = return_index_cv_params(full_proc_traj_data_in, cv_folds_in)
    
    pred_eds = [] 
    pred_mds = [] 
    pred_traces = []
    pred_dets = []
    
    lambda_params_in = {'ml': 1, 'physics': 0}
    
    for jdx, X_mat_train in enumerate(train_X_mats_cv):
        y_vec_train = train_y_vecs_cv[jdx]
        X_mat_test = test_X_mats_cv[jdx]
        y_vec_test = test_y_vecs_cv[jdx]
        physics_metadata_train = physics_metadata_train_dicts_cv[jdx]
        physics_metadata_test = physics_metadata_test_dicts_cv[jdx]
   
        nn_model = BNKF(input_size=X_mat_train.shape[1], output_size=y_vec_train.shape[1], hidden_layer_size=64, sensor_noise_level=sensor_noise_in,
             prior_sigma_lay=0.01, lambda_params=lambda_params_in, epochs=epochs_in, bias_in=True)
        nn_model.fit(X_mat_train, y_vec_train, physics_metadata_train)
        nn_state_est_vecs, nn_state_cov_mats = nn_model.predict(X_mat_test, physics_metadata_test, apply_filter_update=False)
        
        ed, md, trace, det = evaluate_model_outputs(nn_state_est_vecs, nn_state_cov_mats, y_vec_test)
        
        pred_eds.append(ed)
        pred_mds.append(md)
        pred_traces.append(trace)
        pred_dets.append(det)
        
        if show_info:
            print(f"FINAL LOSS k={index}: {nn_model.loss_values[-1]}")
        if single_split:
            break
    return pred_eds, pred_mds, pred_traces, pred_dets

## REGULAR TESTING

## ORIGINAL TRAIN, TEST ORIGINAL

In [7]:
# %%time
# sensor_noise_types = ['low', 'mid', 'high']
# folds = 2
# epochs = 5000
# 
# for sensor_noise_level in sensor_noise_types:
#     path_3 = f"../../dataset/dataframe-readins/full_traj_data_proc_noise_{sensor_noise_level}.pkl"
#     if os.path.exists(path_3):
#         with open(path_3, "rb") as c:
#             full_proc_traj_data = pickle.load(c)
# 
#     cv_folds = construct_dict_folds(full_proc_traj_data, k=folds)
#     out_pred_eds, out_pred_mds, out_pred_uncers, out_pred_dets = cross_validate_pinn_model(full_proc_traj_data, cv_folds, sensor_noise_level, epochs_in=epochs)
#     
#     print(f"CV RESULTS ON NOISE LEVEL: {sensor_noise_level.upper()}")
#     print("\n")
#     print(f"AVG ED: {np.mean(out_pred_eds)}")
#     print(f"AVG MD: {np.mean(out_pred_mds)}")
#     print(f"AVG UNCER: {np.mean(out_pred_uncers)}")
#     print(f"AVG DET: {np.mean(out_pred_dets)}")

## AUGMENT TRAIN, TEST AUGMENT

In [8]:
%%time
sensor_noise_types = ['low', 'mid', 'high']
folds = 5
epochs = 1000

for sensor_noise_level in sensor_noise_types:
    aug_path_in = f"../../dataset/dataframe-readins/cv/augmented/full_traj_data_proc_noise_{sensor_noise_level}.pkl"

    if os.path.exists(aug_path_in):
        with open(aug_path_in, "rb") as e:
            aug_proc_traj_data = pickle.load(e)

    cv_folds = construct_dict_folds(aug_proc_traj_data, k=folds)
    out_pred_eds, out_pred_mds, out_pred_traces, out_pred_dets = cross_validate_pinn_model(aug_proc_traj_data, cv_folds, sensor_noise_in=sensor_noise_level, epochs_in=epochs, single_split=True)

    print(f"CV RESULTS ON NOISE LEVEL: {sensor_noise_level.upper()}")
    print("\n")
    print(f"AVG ED: {np.mean(out_pred_eds)}")
    print(f"AVG MD: {np.mean(out_pred_mds)}")
    print(f"AVG TRACE: {np.mean(out_pred_traces)}")
    print(f"AVG DET: {np.mean(out_pred_dets)}")


 Epoch: 1000/1000 | Loss:   1.94 

CV RESULTS ON NOISE LEVEL: LOW


AVG ED: 1.3584184646606445
AVG MD: 1.5907580337062348
AVG TRACE: 2.870838165283203
AVG DET: 1.3964189416971655


 Epoch: 1000/1000 | Loss:   6.44 

CV RESULTS ON NOISE LEVEL: MID


AVG ED: 4.722892761230469
AVG MD: 5.488800250954494
AVG TRACE: 3.0032615661621094
AVG DET: 1.5446388321239803


 Epoch: 1000/1000 | Loss:  20.38 

CV RESULTS ON NOISE LEVEL: HIGH


AVG ED: 8.599570274353027
AVG MD: 5.817453221991461
AVG TRACE: 7.82319974899292
AVG DET: 11.1135072748194
CPU times: user 3h 55min 18s, sys: 1h 6min 45s, total: 5h 2min 4s
Wall time: 2h 10min


## ONE MODEL FOR ALL

In [9]:
# %%time
# do = True
# if do:
#     sensor_noise_types = ['low', 'mid', 'high']
#     folds = 5
#     consolidated_dict = {}
#     for index, sensor_noise_level in enumerate(sensor_noise_types):
#         aug_path_in = f"../../dataset/dataframe-readins/cv/augmented/full_traj_data_proc_noise_{sensor_noise_level}.pkl"
# 
#         if os.path.exists(aug_path_in):
#             with open(aug_path_in, "rb") as e:
#                 aug_proc_traj_data = pickle.load(e)
#         num_traj = len(aug_proc_traj_data)
# 
#         if index == 1:
#             start_idx = num_traj+1
#             end_idx = start_idx + num_traj
#             new_keys = range(start_idx, end_idx)
#             reset_dict = dict(zip(new_keys, aug_proc_traj_data.values()))
#             consolidated_dict.update(reset_dict)
#         elif index == 2:
#             start_idx = (num_traj*2)+2
#             end_idx = start_idx + num_traj
#             new_keys = range(start_idx, end_idx)
# 
#             reset_dict = dict(zip(new_keys, aug_proc_traj_data.values()))
#             consolidated_dict.update(reset_dict)
#         else:
#             consolidated_dict.update(aug_proc_traj_data)
# 
#     cv_folds = construct_dict_folds(consolidated_dict, k=folds)

In [10]:
# out_pred_eds, out_pred_mds, out_pred_traces, out_pred_dets = cross_validate_pinn_model(consolidated_dict, cv_folds, epochs_in=1000, single_split=False)
# 
# print(f"CV RESULTS AVERAGED ACROSS ALL NOISE LEVELS")
# print("\n")
# print(f"AVG ED: {np.mean(out_pred_eds)}")
# print(f"AVG MD: {np.mean(out_pred_mds)}")
# print(f"AVG TRACE: {np.mean(out_pred_traces)}")
# print(f"AVG DET: {np.mean(out_pred_dets)}")